In [1]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split

import pandas as pd
from datasets import load_dataset, Dataset

C:\Users\raden\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

df = pd.read_csv('srpski.csv', sep='\t')
df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])

df['label'] = df['Autor'].astype('category').cat.codes
df = df.drop(columns=['Autor'])
df = df.rename(columns={'Tekst':'text'})

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 110
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 28
    })
})

In [3]:
model_id = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1' 
model = SetFitModel.from_pretrained(model_id)


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/345 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [4]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={'text':"text", "label": "label"}
)


C:\Users\raden\AppData\Local\Temp\ipykernel_24112\3111768313.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/110 [00:00<?, ? examples/s]

In [5]:
def hyperparameter_search_function(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32])    }


In [7]:
trainer.train()
metrics_train = trainer.evaluate(dataset=dataset['train']) # evaluacija trening skupa
metrics_eval = trainer.evaluate() #evaluacija test skupa

***** Running training *****
  Num examples = 275
  Num epochs = 1
  Total optimization steps = 275
  Total train batch size = 16


Step,Training Loss


Applying column mapping to the evaluation dataset
***** Running evaluation *****
***** Running evaluation *****


In [8]:
print("Results on the training set:")
for key, value in metrics_train.items():
    print(f"{key}: {value}")

print("\nResults on the evaluation set:")
for key, value in metrics_eval.items():
    print(f"{key}: {value}")

Results on the training set:
accuracy: 1.0

Results on the evaluation set:
accuracy: 0.2857142857142857
